In [509]:
import json
import pandas as pd

In [510]:
with open("blocks(attri).json") as infile:
    data = json.load(infile)

In [511]:
cbDict = {}
blocks = {}
jsDict = {}

In [512]:
def addEdge(enTT_s1,enTT_s2):
    for ei in enTT_s1:
        blocks["s1:"+str(ei)] = blocks["s1:"+str(ei)] + 1 if "s1:"+str(ei) in blocks.keys() else 1  
        for ej in enTT_s2:
            blocks["s2:"+str(ej)] = blocks["s2:"+str(ej)] + 1 if "s2:"+str(ej) in blocks.keys() else 1  
            cbDict[("s1:"+str(ei),"s2:"+str(ej))] = cbDict[("s1:"+str(ei),"s2:"+str(ej))] + 1 if ("s1:"+str(ei),"s2:"+str(ej)) in cbDict.keys() else 1

In [513]:
# cbDict

In [514]:
def js():
    for k1,k2 in cbDict.keys():
        jsDict[(k1,k2)] = cbDict[(k1,k2)] / (blocks[k1] + blocks[k2] - cbDict[(k1,k2)])

In [515]:
for d in data:
    enTT_s1 = list(d.values())[0]["s1"]
    enTT_s2 = list(d.values())[0]["s2"]
    addEdge(enTT_s1,enTT_s2)

In [516]:
js()

In [517]:
# jsDict

In [518]:
def pruning(jsDict):
    dic_len = len(jsDict.items())
    threshold = sum(list(jsDict.values())) / dic_len
    return [i[0] for i in sorted(jsDict.items(),key=lambda x : x[1],reverse=True) if i[1] > threshold]

In [519]:
first = pd.read_csv("first.csv",delimiter=";")
second = pd.read_csv("second.csv",delimiter=";")

In [520]:
first.index = first.id
first = first.drop("id",1)

In [521]:
second.index = second.id
second = second.drop("id",1)

In [522]:
# len(pruning(jsDict))

In [523]:
def match(descri1,descri2,threshold):
    def getTokens(descriX):
        return [x for p in [i.split() for i in descriX.values if i != "NONE"] for x in p]
    def jaccard(x,y):
        return len(set(x).intersection(set(y))) / len(set(x+y))
    tokens1 = getTokens(descri1)
    tokens2 = getTokens(descri2)
    return True if jaccard(tokens1,tokens2) >= threshold else False

In [524]:
tmp = pruning(jsDict)
jSim = [jsDict[(i[0],i[1])] for i in tmp]
jCol1 = [i[0][3:] for i in tmp]
jCol2 = [i[1][3:] for i in tmp]
header = ["c","entity_set_2","JS(ei,ej)","predMatch","realMatch"]
jPredMatch = []
jRealMatch = []

In [525]:
show = pd.DataFrame(jCol1,jCol2)

In [526]:
show["entity_set_2"] = show.index
show = show.rename(columns={0:"entity_set_1"})
show.index = range(len(tmp))
show["edge_JS(ei,ej)"] = jSim
show["predMatch"] = 0
show["realMatch"] = 0

In [528]:
def cmp(row):
    index1,index2 = int(row["entity_set_1"]),int(row["entity_set_2"])
    pred = match(first.loc[index1,:],second.loc[index2,:],0.3)
    real = pred and index1 == index2
    row["predMatch"] = pred
    row["realMatch"] = real
    return row

In [529]:
show = show.apply(lambda x:cmp(x),axis=1)

In [530]:
# show.head(15)

In [531]:
print("Pairs Quality(PQ): ",sum(show.predMatch)/len(tmp))

Pairs Quality(PQ):  0.018643395477984926


In [532]:
print("Pairs Completeness(PC): ",sum(show.realMatch)/90)

Pairs Completeness(PC):  0.8555555555555555
